In [1]:
import numpy as np

import scipy.stats as stats
import matplotlib.pyplot as plt

import time

#For inline plotting 
%matplotlib inline                 
%config InlineBackend.figure_format = 'svg'


plt.style.use("seaborn-v0_8-white")

### **American Option Pricing with Binomial Trees**
-------

In contrast with the European vanilla options, American options give the holder the right to exercise it at any time before expiry $t \leq T$ if it is advantageous to do so. 

**Remark:** With the assumption that interest rates are positive, it is *never optimal to exercise an American Call Option -on a non-dividend paying underlying- before the expiry*. This is because at the time of exercise we pay $K$ amount to buy the stock, which could be otherwise invested in a money market account with interest. Furthermore, since the underlying does not pay dividends, stock price won't have a chance to drop in value during the contract, rendering early exercise choice of American Options not advantageous over the European Vanilla Options. 

In summary, there is no incentive to exercise early an American Call Option and so there is no added value from the American feature. Since the right to early exercise is not executed in the optimal strategy, we have 

$$
\textrm{American Calls} = \textrm{European Calls} \quad\quad\quad \textrm{for non-dividend stocks}
$$

provided that interest rates are positive, $r > 0$. In case the latter is negative (and the underlying pays dividends), one can find scenarios making the early exercise property appealing. 

In what follows, we will therefore focus on the pricing of American Put Options using Binomial tree method.

- For American Puts, we can directly see the benefit of early exercise as it guarantees us to receive $K$ amount from the sell of the stock now, from which we can earn interest over time as long as $r > 0$. Here, it does not matter if the underlying pays dividend or not, in fact dividend payments are acting towards our advantage by reducing the price of the stock. 

- Therefore, in contrast with the European counterpart, we need to take into account the decision of “Should I exercise the option now, or is it better to hold?” at each node in the binomial tree. Such a decision can be said to influence the value of the option in nodes that precedes it, when we use a backward induction process along the binomial tree. 

For this purpose, we consider a binomial tree that is discrete time, recombining model of the underlying stock price;

- $S_0$: initial stock price
- $K$: strike price
- $u$: up factor
- $d = 1/u$: down factor ($1/u$ to ensure recombination)
- $r$: risk-free rate
- $\Delta t$: unit time step on the tree
- $N$: number of time steps 

Using the risk-neutrality condition at a unit binomial tree, we can also obtain the risk-neutral probability, 

$$
\mathbb{E}[S_{t + \Delta t}] = S_t \, \mathrm{e}^{r \Delta t}\quad\quad \longrightarrow \quad\quad q = \frac{\mathrm{e}^{r \Delta t} - d}{u - d}
$$




### Binomial Tree Model
-----

We label each node in the binomial tree via $(i,j)$: 

- $i$: time step from $0$ to $N$
- $j$: number of up moves from $0$ to $i$

The stock price at any node in the tree is then described by 

$$
S^{\,j}_{\,\,i} = S_0\, u^{j}\, d^{\,i-j} 
$$

For a given depth of the tree specified by the number of time steps $N$, we can then compute the option value at the expiry using the stock value at that time,

$$
C^{\, j}_{\,\,N} = \textrm{max}(0, K - S^{\,j}_{\,\,N})
$$

which corresponds to would have been put option value without early exercise. Therefore, for a given $j$ node at the final time step, these values is equivalent to the European put option values for possible choices of stock price values. If we wanted to know the price of a European option on this tree, we would simply trace back these values back in time through discounting the expected values that come out from each node, as implied by the risk-neutral framework. However, now we have the extra power to decide whether we should exercise the option prior to expiry. The natural criterion for this decision depends on whether immediate exercise value of the option is greater than the continuation value (e.g its discounted expected future value). If yes, we treat the value of the option as its instantaneous value otherwise its value is equal to the standard European value. 

Therefore, for the rest of the time nodes $i = N-1,\dots 0$, we compute the value of the option as the maximum between the intrinsic value and continuation value: 

$$
C^{\, j}_{\,\,i} = \textrm{max}\left(K - S^{\,j}_{\,\,i},\, \mathrm{e}^{-r \Delta t} \left[q\, C^{\, j + 1}_{\,\,\,i + 1} + (1-q)\, C^{\, j}_{\,\,i + 1} \right]\right)
$$



In [5]:
S_in = 100 # initial stock price
K = 100 # strike price
T = 1 # maturity in years
r = 0.06 # risk-free rate

# up jump move factor and Number of time steps
u = 1.1
N = 3

In [8]:
# import binomial tree model of the underlying

from models import binomial_tree_model

binom_model = binomial_tree_model(u, r, T, N)

In [9]:
# import american_option class

from options import american_option

us_option = american_option(binom_model, S_in, K, T, is_call=False)
us_price = us_option.price()
print(f"American put price: {us_price:.5f} $")

American put price: 4.65459 $


We can simply downgrade our `american_option` class to cover the standard European call options which I implement in `european_option` class

In [10]:
from options import european_option

eu_option = european_option(binom_model, S_in, K, T, is_call=True)
eu_price = eu_option.price()
print(f"European call price: {eu_price:.5f} $")

European call price: 10.14574 $


### **Pricing Barrier Options with Binomial Tree**
------

Barrier options differ from their European and American counterpart in that they are *path dependent*. 

In particular, Barrier options are like standard options, but with an added condition:

- The option only becomes active (knock-in) or void (knock-out) if the underlying asset touches or breaches a barrier level during its life.

Most such types of options are European style, but their American version does also exist. Here we focus on the former types which can be classified as follows



| **Option Type**  | **Barrier Condition**                                                              | **Payoff at Maturity**                                 |
| ---------------- | ---------------------------------------------------------------------------------- | ------------------------------------------------------ |
| **Up-and-in**    | Option **only becomes active** if the asset price **ever rises above** the barrier | If barrier breached: standard European payoff; else: 0 |
| **Up-and-out**   | Option is **knocked out** (voided) if price **ever rises above** the barrier       | If barrier not breached: standard payoff; else: 0      |
| **Down-and-in**  | Option **only becomes active** if price **ever falls below** the barrier           | If barrier breached: standard European payoff; else: 0 |
| **Down-and-out** | Option is **knocked out** if price **ever falls below** the barrier                | If barrier not breached: standard payoff; else: 0      |



For example;

Let’s say you have a European call with strike $K = 100$ and the final stock price is $S_T = 110$:

- Up-and-in, barrier $H = 120$ ---> if $S_t$ never hits $120$, pay-off is $0$.

- Up-and-out, barrier $H = 120$ ---> if $S_t$ never hits $120$, pay-off is $110 - 100 = 10$.

- Down-and-in, barrier $H= 80$ ---> if $S_t$ fell below $80$ at some point, pay-off is $110 - 100 = 10$.

- Down-and-out, barrier $ H= 80$ ---> if $S_t$ never falls below $80$ , pay-off is $110 - 100 = 10$.

These examples correspond to a call type barrier options but similar arguments also apply to put types.


Due to path dependence, we therefore need to check/track the entire path of the barrier options $S_t \in (0,T]$ whether the barrier $H$ is ever breached. On a binomial tree setup, we can for example perform a discrete monitoring at each time step for this purpose.   


We can thus mathematically describe the pay-offs contingent on the extra conditions dependening on the nature of the contract. For example, for an Put option we have the following pay-offs; 


| **Option Type**  | **Payoff at Maturity**                                 |
| ---------------- | ------------------------------------------------------ | 
| **Up-and-in**    | $(K - S)^{+} \,\, \mathbb{I}(\textrm{max}_{\forall t} S_t \geq H)$ |
| **Up-and-out**   | $(K - S)^{+} \,\, \mathbb{I}(\textrm{max}_{\forall t} S_t < H)$     |
| **Down-and-in**  | $(K - S)^{+} \,\, \mathbb{I}(\textrm{min}_{\forall t} S_t \leq H)$  |
| **Down-and-out** | $(K - S)^{+} \,\, \mathbb{I}(\textrm{min}_{\forall t} S_t > H)$      |

using which we can guess the call equivalents. As a side note, the table above actually suggests a nice relation 

$$
\textrm{Vanilla option} = \textrm{Knock-in option} + \textrm{Knock-out option} 
$$

The condition above actually suggest something obvious: the European vanilla option will be always worth at least as much as a barrier option or more. This is obvious as the barrier option contract has an additional constraint that its European counterpart does not have. 

In [15]:
from options import barrier_option

# extra contract barrier conditions 
H = 125
option_type = 'call'
barrier_type = 'up-and-out'

b_option = barrier_option(binom_model, S_in, K, H, T, option_type=option_type, barrier_type = barrier_type)
b_price = b_option.price()
print(f"Barrier {barrier_type} {option_type} price: {b_price:.5f} $")

Barrier up-and-out call price: 4.00027 $
